In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist, fashion_mnist, cifar10
from tensorflow.keras.utils import to_categorical
# Load the dataset
(dataset_train, labels_train), (dataset_test, labels_test) =  fashion_mnist.load_data()  

# Preprocess the data
dataset_train = dataset_train.astype('float32') / 255
dataset_test = dataset_test.astype('float32') / 255

if len(dataset_train.shape) == 3:
    dataset_train = np.expand_dims(dataset_train, axis=-1)
    dataset_test = np.expand_dims(dataset_test, axis=-1)

# One-hot encode the labels
labels_train = to_categorical(labels_train, 10)
labels_test = to_categorical(labels_test, 10)


In [2]:
!pip install -U tensorflow-addons
!pip install -U tensorflow-hub


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
import tensorflow_hub as hub
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator



# Function to preprocess the dataset
def preprocess_dataset(dataset):
    dataset_rgb = np.repeat(dataset, 3, axis=-1)  # Convert to RGB
    return dataset_rgb

# Preprocess the dataset without resizing
preprocessed_train = preprocess_dataset(dataset_train)
preprocessed_test = preprocess_dataset(dataset_test)

# Functions to create simple neural network and LeNet-5 models
def simple_nn(input_shape, num_classes):
    model = models.Sequential([
        layers.Flatten(input_shape=input_shape),
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

def lenet_5(input_shape, num_classes):
    model = models.Sequential([
        layers.Conv2D(6, (5, 5), activation='relu', input_shape=input_shape),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Conv2D(16, (5, 5), activation='relu'),
        layers.MaxPooling2D(pool_size=(2, 2)),
        layers.Flatten(),
        layers.Dense(120, activation='relu'),
        layers.Dense(84, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Train and evaluate simple neural network and LeNet-5 models
input_shape = preprocessed_train.shape[1:]
num_classes = 10

for model_architecture in [simple_nn, lenet_5]:
    model = model_architecture(input_shape, num_classes)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(preprocessed_train, labels_train, epochs=1, batch_size=1, validation_split=0.2)
    model_score = model.evaluate(preprocessed_test, labels_test)
    print(f"{model_architecture.__name__} Test Loss:", model_score[0])
    print(f"{model_architecture.__name__} Test Accuracy:", model_score[1])


/usr/local/lib/python3.10/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


 3423/48000 [=>............................] - ETA: 5:29 - loss: 0.8700 - accuracy: 0.6784

In [ ]:
import tensorflow_hub as hub
import tensorflow_addons as tfa
from tensorflow.keras.preprocessing.image import ImageDataGenerator


load the pretrained models and apply transfer learning:

In [ ]:
import numpy as np
import cv2

# Function to preprocess the dataset
def preprocess_dataset(dataset, target_size):
    dataset_rgb = np.repeat(dataset, 3, axis=-1)  # Convert to RGB
    resized_dataset = np.array([cv2.resize(img, target_size) for img in dataset_rgb])  # Resize the images
    return resized_dataset

# Preprocess the dataset
target_size = (224, 224)  # Use (224, 224) for EfficientNet and (160, 160) for BiT
preprocessed_train = preprocess_dataset(dataset_train, target_size)
preprocessed_test = preprocess_dataset(dataset_test, target_size)

# Function to create a model with transfer learning
def create_tl_model(model_url, input_shape, num_classes):
    base_model = hub.KerasLayer(model_url, trainable=True)
    model = models.Sequential([
        layers.InputLayer(input_shape),
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# EfficientNet
efficientnet_url = "https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1"
efficientnet_model = create_tl_model(efficientnet_url, target_size+(3,), num_classes)
efficientnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

efficientnet_model.fit(preprocessed_train, labels_train, epochs=1, batch_size=1, validation_split=0.2)


# BiT
bit_url = "https://tfhub.dev/google/bit/m-r50x1/1"
bit_model = create_tl_model(bit_url, target_size+(3,), num_classes)
bit_model.compile(optimizer=tfa.optimizers.AdamW(learning_rate=0.001, weight_decay=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

bit_model.fit(preprocessed_train, labels_train, epochs=1, batch_size=1, validation_split=0.2)

c) Image augmentation:

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Train the model with augmented data
augmented_model = simple_nn(input_shape, num_classes)  # You can replace this with another model like lenet_5 or the transfer learning models
augmented_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

augmented_model.fit(datagen.flow(dataset_train, labels_train, batch_size=1), epochs=1, validation_data=(dataset_test, labels_test))


d) State-of-the-art image models - MLP-Mixer:



In [ ]:
!pip install -U timm
import timm
# Create MLP-Mixer model
def create_mlp_mixer(input_shape, num_classes):
    model = timm.create_model("mixer_b16_224", pretrained=True, num_classes=num_classes, in_chans=1)
    return model

# Train MLP-Mixer
mlp_mixer_model = create_mlp_mixer(input_shape, num_classes)
mlp_mixer_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

mlp_mixer_model.fit(dataset_train, labels_train, epochs=1, batch_size=32, validation_split=0.2)
